# WESM Price Prediction - ETL/Preprocessing

## Setup and "Loader"

This part sets up the method for loading and concatenating of *.csv* files.

In [1]:
import pandas as pd
import glob
import os

# Paths
PATH_GWAP = "./raw_data/GWAP/"
PATH_RTD = "./raw_data/RTD_Regional/"
PATH_OUTAGES = "./raw_data/Outages/"

# Connects extracted CSV files
def load_and_concatenate_csvs(path):
    files = glob.glob(os.path.join(path, "*.csv"))
    print(f"File Count in {path}: {len(files)}")

    dfs = []
    for file in files:
        df = pd.read_csv(file)
        dfs.append(df)

    concatenated_df = pd.concat(dfs, axis=0, ignore_index=True)
    print(f"Concatenated DataFrame Shape from {path}: {concatenated_df.shape}")
    
    return concatenated_df

## GWAP Processing

For this portion, we load all GWAP (Generation Weighted Average Price) CSV files into a single DataFrame, `df_gwap`. For the purposes of this project, we retain only the rows where `REGION_NAME` is `CLUZ`, restricting the dataset to Luzon GWAP values.

In [2]:
df_gwap = load_and_concatenate_csvs(PATH_GWAP)
df_gwap = df_gwap[df_gwap["REGION_NAME"] == "CLUZ"].copy()
df_gwap["datetime"] = pd.to_datetime(df_gwap["TIME_INTERVAL"], format='mixed', dayfirst=False)
df_gwap = df_gwap[["datetime", "GWAP"]].sort_values("datetime")
df_gwap = df_gwap.drop_duplicates(subset=['datetime']) # just in case

print(f"Final GWAP DataFrame Shape: {df_gwap.shape}")
print(df_gwap.head())

File Count in ./raw_data/GWAP/: 90
Concatenated DataFrame Shape from ./raw_data/GWAP/: (129690, 6)
Final GWAP DataFrame Shape: (25920, 2)
               datetime       GWAP
366 2025-10-28 00:05:00  2800.5685
371 2025-10-28 00:10:00  2985.7148
314 2025-10-28 00:15:00  2899.6850
377 2025-10-28 00:20:00  2992.3605
416 2025-10-28 00:25:00  2949.7466


## RTD Processing

This portion loads all RTD Regional (Real-Time Dispatch Regional) CSV files, retaining only rows where `REGION_NAME` is `CLUZ`.

Next, rows where `COMMODITY_TYPE` is `En` are extracted and stored in `df_energy`, as this commodity represents actual energy in the RTD market. The retained columns are:
- `datetime`
- `MKT_REQT`, renamed to `energy_demand_mw`
- `GENERATION`, renamed to `energy_supply_mw`
- Additionally, an engineered feature, `energy_shortage_mw`, is added. This represents the difference between energy demand and supply and serves as an indicator of potential load shedding or brownouts.

All remaining rows where `COMMODITY_TYPE != En` are treated as reserve commodities and stored in `df_reserves`. These rows are grouped by `datetime`, with the following aggregated columns retained:
- `datetime`
- `MKT_REQT`, summed and renamed to `reserve_demand_mw`
- `GENERATION`, summed and renamed to `reserve_supply_mw`

As a final step, `df_energy` and `df_reserves` are merged on `datetime` to produce `df_X`, which contains the final set of RTD-derived features used in the analysis.

In [3]:
df_rtd = load_and_concatenate_csvs(PATH_RTD)
df_rtd = df_rtd[df_rtd["REGION_NAME"] == "CLUZ"].copy()
df_rtd["datetime"] = pd.to_datetime(df_rtd["TIME_INTERVAL"], format='mixed', dayfirst=False)

# Extract Energy (Demand & Supply)
# Commodity Type "En" refers to energy
df_energy = df_rtd[df_rtd["COMMODITY_TYPE"] == "En"].copy()
df_energy = df_energy[["datetime", "MKT_REQT", "GENERATION"]]
df_energy.columns = ["datetime", "energy_demand_mw", "energy_supply_mw"]

# Engineered Feature: Energy Shortage (Demand - Supply)
# If > 0, load shedding / brownouts likely
df_energy['energy_shortage_mw'] = df_energy['energy_demand_mw'] - df_energy['energy_supply_mw']

# Extract Reserves (Safety Net)
# Anything that is not "En" are reserves
df_reserves = df_rtd[df_rtd["COMMODITY_TYPE"] != "En"].copy()

# Group reserves by datetime then sum them up
df_reserves = df_reserves.groupby("datetime")[["MKT_REQT", "GENERATION"]].sum().reset_index()
df_reserves.columns = ["datetime", "reserve_demand_mw", "reserve_supply_mw"]

# Note: There used to be a reserve_shortage column but after EDA, it was found to be mostly zeros (apart from one point) so it was removed.

# Merge them back
df_X = pd.merge(df_energy, df_reserves, on="datetime", how="left")
df_X = df_X.fillna(0)  # Fill NaNs with 0

print(f"Final RTD Features DataFrame Shape: {df_X.shape}")
print(df_X.head())

File Count in ./raw_data/RTD_Regional/: 90
Concatenated DataFrame Shape from ./raw_data/RTD_Regional/: (388815, 13)
Final RTD Features DataFrame Shape: (25915, 6)
             datetime  energy_demand_mw  energy_supply_mw  energy_shortage_mw  \
0 2025-10-28 00:05:00           9043.68           9095.22              -51.54   
1 2025-10-28 00:10:00           9002.76           9038.49              -35.73   
2 2025-10-28 00:15:00           8975.73           9026.83              -51.10   
3 2025-10-28 00:20:00           9027.64           9012.70               14.94   
4 2025-10-28 00:25:00           9003.70           9001.07                2.63   

   reserve_demand_mw  reserve_supply_mw  
0             1495.0             1495.0  
1             1495.0             1495.0  
2             1495.0             1495.0  
3             1495.0             1495.0  
4             1495.0             1495.0  


## Outage Processing

This portion loads all Outages CSV files into a single DataFrame, `df_outages`.

First, rows with missing `RESOURCE_NAME` are dropped, as these entries cannot be reliably associated with a specific generator.

Next, the dataset is filtered to include only Luzon plants, identified by the first two digits of `RESOURCE_NAME` (`01`, `02`, `03`). A new column, `prefix`, is created for this purpose, and only rows with Luzon prefixes are retained.

A `datetime` column is then created from the `RUN_TIME` column to enable proper time-series alignment.

Finally, the outages are aggregated by `datetime`, resulting in a new DataFrame, `df_out_count`, which contains the total count of Luzon outages per time interval. This provides a concise, time-aligned representation of outages for further analysis and integration with RTD and GWAP data.

In [4]:
df_outages = load_and_concatenate_csvs(PATH_OUTAGES)

# Drop rows with missing Resource Name
initial_len = len(df_outages)
df_outages = df_outages.dropna(subset=['RESOURCE_NAME'])
print(f"Dropped {initial_len - len(df_outages)} rows with missing Resource Name.")

# Filter to Luzon plants only (Prefix 01-03)
df_outages['prefix'] = df_outages['RESOURCE_NAME'].astype(str).str[:2]
luzon_prefixes = ['01', '02', '03']
df_outages = df_outages[df_outages['prefix'].isin(luzon_prefixes)].copy()
print(f"Filtered for Luzon (Prefixes 01-03). Remaining rows: {len(df_outages)}")

df_outages['datetime'] = pd.to_datetime(df_outages['RUN_TIME'], format='mixed', dayfirst=False)
df_out_count = df_outages.groupby('datetime').size().reset_index(name='outage_count') # count of outages per datetime

print(f"Final Outages DataFrame Shape: {df_out_count.shape}")
print(df_out_count.head())

File Count in ./raw_data/Outages/: 90
Concatenated DataFrame Shape from ./raw_data/Outages/: (345237, 7)
Dropped 90 rows with missing Resource Name.
Filtered for Luzon (Prefixes 01-03). Remaining rows: 117926
Final Outages DataFrame Shape: (25915, 2)
             datetime  outage_count
0 2025-10-28 00:05:00             4
1 2025-10-28 00:10:00             4
2 2025-10-28 00:15:00             4
3 2025-10-28 00:20:00             4
4 2025-10-28 00:25:00             4


## Merge

This portion merges the previously processed GWAP, RTD, and Outages datasets to create a single, time-aligned dataset for analysis.

First, `df_gwap` (Luzon GWAP prices) is merged with `df_X` (RTD features including energy demand/supply, reserves, and energy shortage) on `datetime` using an inner join. This ensures that only timestamps present in both datasets are retained.

Next, the outage data (`df_out_count`) is merged using a left join, adding the `outage_count` feature to the dataset. Missing values are filled with 0, indicating that no outages occurred at those timestamps.

To capture temporal dependencies in electricity prices, lag features are created from GWAP:
- `GWAP_Lag_1` → GWAP value 5 minutes ago  
- `GWAP_Lag_12` → GWAP value 1 hour ago  
- `GWAP_Lag_288` → GWAP value at the same time yesterday (24 hours ago)  

Rows with missing values, resulting from the lag feature creation, are dropped.

The final dataset is saved to `final_dataset.csv` and contains the following features for each timestamp:
- GWAP
- Energy demand, supply, and shortage  
- Reserve demand and supply  
- Outage count  
- Lagged GWAP values

This dataset is now fully prepared for exploratory data analysis, visualization, or predictive modeling.

In [5]:
# Merge GWAP with RTD features
final_df = pd.merge(df_gwap, df_X, on="datetime", how="inner")

# Merge with Outages data
final_df = pd.merge(final_df, df_out_count, on="datetime", how="left")
final_df['outage_count'] = final_df['outage_count'].fillna(0)  # Fill NaNs with 0

# Time Lags (for autocorrelation)
final_df['GWAP_Lag_1'] = final_df['GWAP'].shift(1)    # 5 mins ago
final_df['GWAP_Lag_12'] = final_df['GWAP'].shift(12)  # 1 hour ago
final_df['GWAP_Lag_288'] = final_df['GWAP'].shift(288) # 24 hours ago (Yesterday same time)

final_df = final_df.dropna()
final_df.to_csv("final_dataset.csv", index=False)

print(f"Final Dataset Shape: {final_df.shape}")
print(f"Date Range: {final_df['datetime'].min()} to {final_df['datetime'].max()}")
print(final_df.head())

Final Dataset Shape: (25627, 11)
Date Range: 2025-10-29 00:05:00 to 2026-01-26 00:00:00
               datetime       GWAP  energy_demand_mw  energy_supply_mw  \
288 2025-10-29 00:05:00  2258.5866           9504.45           9705.25   
289 2025-10-29 00:10:00  2913.4257           9483.49           9648.89   
290 2025-10-29 00:15:00  2883.5764           9434.35           9627.98   
291 2025-10-29 00:20:00  2888.6233           9417.62           9614.19   
292 2025-10-29 00:25:00  2887.9553           9379.48           9575.17   

     energy_shortage_mw  reserve_demand_mw  reserve_supply_mw  outage_count  \
288             -200.80             1428.0             1428.0           4.0   
289             -165.40             1428.0             1428.0           4.0   
290             -193.63             1428.0             1428.0           4.0   
291             -196.57             1428.0             1428.0           4.0   
292             -195.69             1428.0             1428.0           